In [1]:
import sys
import pandas as pd
import numpy as np
import ipywidgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import warnings
import plotly.express as px
import plotly.graph_objects as go

In [2]:
CODE_PATH = r"C:\personal\proj_university\seminar\eegtools"
sys.path.insert(0, CODE_PATH)
import data_ingestion, plot, analysis

In [3]:
MAT_PATH = r"C:\personal\proj_university\seminar\Data for example\Itamar_Data.mat"
DATA_NAME = "Data_All"
data = data_ingestion.MatIngest(filename=MAT_PATH, data_name=DATA_NAME).create_data_obj()

In [4]:
# Extraction of the timeline as a constant (512,) vector
TIMELINE = data['time']
MU_STR = '\u03BC'

In [5]:
# function to get Control/Asd subject's idx
def get_groups_idx(data, cont_name='Control', asd_name='ASD', label_name='group'):
    cont_ind = np.where(data[label_name]==cont_name)[0]
    asd_ind = np.where(data[label_name]==asd_name)[0]
    return {cont_name:cont_ind, asd_name:asd_ind}

# t is a vector
def get_t_idx(timeline, t):
    vf = np.vectorize(lambda t: np.argmin(np.abs(timeline-t)))
    return vf(t) # idxes for corresponding t (same length)


def get_group_partial_data(data, group, trial_range, block_range):
    idx = get_groups_idx(data)[group]
    return data['s2'][idx,:,trial_range[0]:trial_range[1],block_range[0]:block_range[1]]

In [6]:
idx_map = get_groups_idx(data)
cont_id = idx_map['Control']
asd_id = idx_map['ASD']

In [7]:
# turn off warnings of mean NaN
warnings.filterwarnings(action="ignore", message="Mean of empty slice", category=RuntimeWarning)

empty_sigs = np.argwhere(np.all(np.isnan(data['s2']), axis=1)) # 3d-array with shape (#subjects,#trials,#blocks)
df_empty = pd.DataFrame(empty_sigs)
# append subID and group colunms
df_empty = pd.concat(
    [df_empty[0].apply(
        lambda i: pd.Series([data['subjects'][i],data['group'][i]])), 
     df_empty[[1,2]]],
    axis=1)

In [8]:
BLOCKS = np.arange(data['s2'].shape[3])
start, stop = get_t_idx(TIMELINE,[70,130])

In [9]:

df = pd.DataFrame({"group":np.repeat("Control",len(BLOCKS)), "block":BLOCKS}).append(
    pd.DataFrame({"group":np.repeat("ASD",len(BLOCKS)), "block":BLOCKS}), ignore_index=True)

def get_first(data, gr, bl):
    sig = get_group_partial_data(data,gr,(0,2),(bl,bl+1))
    sig = np.nanmean(sig,axis=2)
    comp = analysis.Component(sig[:,start:stop,:], "N")
    peaks = comp.peak(axis=1)
    return np.nanmean(peaks)

def get_second(data, gr, bl):
    sig = get_group_partial_data(data,gr,(2,10),(bl,bl+1))
    sig = np.nanmean(sig,axis=2)
    comp = analysis.Component(sig[:,start:stop,:], "N")
    peaks = comp.peak(axis=1)
    return np.nanmean(peaks)
    

In [10]:
df['first'] = df.apply(lambda row: get_first(data,row['group'],row['block']), axis=1)
df['second'] = df.apply(lambda row: get_second(data,row['group'],row['block']), axis=1)

In [15]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, subplot_titles=("Control", "ASD"))

fig.add_trace(
    go.Scatter(x=BLOCKS, y=df.loc[df["group"]=="Control", "first"],
               name="trials 1-2", marker=dict(color="firebrick"), mode="lines+markers"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=BLOCKS, y=df.loc[df["group"]=="Control", "second"],
               name="trials 3-10",marker=dict(color="gold"), mode="lines+markers"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=BLOCKS, y=df.loc[df["group"]=="ASD", "first"],
               name="asd_first", marker=dict(color="firebrick"), mode="lines+markers", showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=BLOCKS, y=df.loc[df["group"]=="ASD", "second"],
               name="asd_second",marker=dict(color="gold"), mode="lines+markers", showlegend=False),
    row=2, col=1
)

fig.update_xaxes(title_text="block")
fig.update_yaxes(range=[0,3],title_text="(-1)*"+MU_STR+"V")
fig.update_layout(title_text="N1 Peak Comparison Before and After Trial 2")
fig.show()

In [17]:
fig = make_subplots(rows=2, cols=1, subplot_titles=("Trials 1-2", "Trials 3-10"))

fig.add_trace(
    go.Scatter(x=BLOCKS, y=df.loc[df["group"]=="Control", "first"],
               name="", marker=dict(color="royalblue"), mode="lines+markers", showlegend=False),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=BLOCKS, y=df.loc[df["group"]=="ASD", "first"],
               name="",marker=dict(color="yellowgreen"), mode="lines+markers", showlegend=False),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=BLOCKS, y=df.loc[df["group"]=="Control", "second"],
               name="", marker=dict(color="royalblue"), mode="lines+markers", showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=BLOCKS, y=df.loc[df["group"]=="ASD", "second"],
               name="",marker=dict(color="yellowgreen"), mode="lines+markers", showlegend=False),
    row=2, col=1
)

fig.update_xaxes(title_text="block")
fig.update_yaxes(range=[0,3],title_text="(-1)*"+MU_STR+"V")
fig.update_layout(title_text="N1 Peak Comparison Before and After Trial 2")
fig.show()

22